In [47]:
# Import packages and libraries
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

In [48]:
# Load training data into dataframe
X_train = pd.read_csv('../Data/Training_Features.csv')
X_train.drop(columns=['date_recorded', 'permit', 'public_meeting'], inplace=True)
# Drop columns with NaN values
X_train.dropna(axis=1, inplace=True)

y_train = pd.read_csv('../Data/Training_Labels.csv')

# Ordinally encoding the target.
y_train.replace({'functional': 1, 'non functional': 0, 'functional needs repair': 2}, inplace=True)

# Merge data into one frame using the ID column
combined_frame = pd.merge(X_train, y_train, on='id')
combined_frame

,id,amount_tsh,gps_height,longitude,latitude,wpt_name,num_private,basin,region,region_code,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,1390,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1
1,8776,0.0,1399,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1
2,34310,25.0,686,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1
3,67743,0.0,263,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,0
4,19728,0.0,0,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,1210,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kilimanjaro,3,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1
59396,27263,4700.0,1212,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Iringa,11,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,1
59397,37057,0.0,0,34.017087,-8.750434,Mashine,0,Rufiji,Mbeya,12,...,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,1
59398,31282,0.0,0,35.861315,-6.378573,Mshoro,0,Rufiji,Dodoma,1,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,1


In [49]:
# Split into train, validation, and test set
train, val, test = np.split(combined_frame.sample(frac=1), [int(0.8*len(combined_frame)), int(0.9*len(combined_frame))])

In [50]:
# Confirm expected results
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

47520 training examples
5940 validation examples
5940 test examples


In [51]:
# Method to convert dataframe to a Tensorflow dataset object
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('status_group')
    df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [52]:
# Set batch size and run method on training data
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

/var/folders/y8/vq429chs6djb0hl0y0zp5fg00000gn/T/ipykernel_24602/1929805004.py:5: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}


In [53]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of tsh:', train_features['amount_tsh'])
print('A batch of targets:', label_batch )

Every feature: ['id', 'amount_tsh', 'gps_height', 'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'region', 'region_code', 'district_code', 'lga', 'ward', 'population', 'recorded_by', 'construction_year', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group', 'status_group']
A batch of tsh: tf.Tensor(
[[   0.]
 [   0.]
 [   0.]
 [   0.]
 [3000.]], shape=(5, 1), dtype=float64)
A batch of targets: tf.Tensor([0 2 2 1 2], shape=(5,), dtype=int64)


In [62]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [63]:
amount_tsh_col = train_features['amount_tsh']
layer = get_normalization_layer('amount_tsh', train_ds)
layer(photo_count_col)

AttributeError: module 'tensorflow.keras.layers' has no attribute 'Normalization'

In [64]:
print(tf.__version__)

2.4.1
